In [1]:
import argparse
import json
from pathlib import Path

import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoModelForCausalLM, AutoTokenizer

/Users/njf/code/frank-llm/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tokenizer = AutoTokenizer.from_pretrained("/nfs/public/hf/models/meta-llama/Meta-Llama-3-8B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("/nfs/public/hf/models/meta-llama/Meta-Llama-3-8B-Instruct",
#     # token='hf_xxx'
#     device_map="auto",
#     trust_remote_code=True,
#     torch_dtype=torch.bfloat16,
#     local_files_only=True
# )

In [4]:
path = Path("../resources/rewriting/examples.json")
with path.open(encoding='utf-8') as f:
    data = json.load(f)

In [27]:
system_content = "'You are a query answering system explaining the reasoning behind the methodology chosen to generate an answer. Summarise the following explanation while using formal language and reducing redundancy. Round population figures two three significant figures - for example, 69112701.18 rounds to 69.1 million, 65343184.0 rounds to 65.3 million."

inputs = []
for example in data[:2]:
    for explanation in example['explanation']:
        user_content = ""
        user_content += f"Question: {example['question']} Explanation: {explanation}."

        messages = [
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": user_content
            }
        ]

        inputs.append(messages)

In [28]:
input_messages = [tokenizer.apply_chat_template(i) for i in inputs]
input_ids = [tokenizer.encode(i, return_tensors="pt") for i in inputs_messages]
input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
outputs = model.generate(input_ids)
responses = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

[[{'role': 'system',
   'content': "'You are a query answering system explaining the reasoning behind the methodology chosen to generate an answer. Summarise the following explanation while using formal language and reducing redundancy. Round population figures two three significant figures - for example, 69112701.18 rounds to 69.1 million, 65343184.0 rounds to 65.3 million."},
  {'role': 'user',
   'content': 'Question: What will be the population of France in 2026? Explanation: The value of the population of France in 2026 is 69247799. HOW: Generated a regression function from times between 2010 and 2018.The predicted value of the population of France in 2026 is 69247799. Retrieved fact(s) from the World Bank and Wikidata knowledge sources.. WHY: Could not find the population of France in 2026. Attempted to infer the required value in 2026 by finding the population of France at other times between 2010 and 2018..'}],
 [{'role': 'system',
   'content': "'You are a query answering syst